In [5]:
# %load crawler_v10x2.py


# In[1]:

from urllib.parse import urlencode
import requests
from bs4 import BeautifulSoup
from lxml import etree
import time
import re
from fake_useragent import UserAgent

import sys


# In[2]:

# 自定義的Error class
class CmpyinfoCrawlerError(Exception):
    # 起始時間sta統一由別人給，但每次會自動抓出end時間，得到目前程式執行多久了
    def __init__(self, prefix, sta=0):
        self.sta = sta
        self.end = time.time()
        self.exe = self.end - self.sta
        self.prefix = prefix
            
    # 印出錯誤訊息之前順便抓一下時間，時間為執行時間
    def __str__(self):
        # 將秒數轉換成可讀的時間格式字串
        def timestr(t):
            d, r = divmod(t, 86400)
            h, r = divmod(r, 3600)
            m, s = divmod(r, 60)
            return "{} days {:02}:{:02}:{:.2f}".format(d, h, m, s)
        
        s = self.prefix + " raised at {} ".format(timestr(self.end-self.sta))
        return s


# In[3]:
# 自定義的proxypool模組
import proxypool


# In[4]:

# 自定義的log class雖然會記錄資訊，但實際使用時不太會看
class back_log:
    def __init__(self, flush=False, flush_threshold=1000, first_line='',log_format='', fname =''):
        # self.queue：用來記錄log
        # self.flush：用來判斷是否要將self.queue的所有紀錄寫到檔案
        # self.flush_threshold：記錄幾筆log之後要寫到檔案
        # self.format：設定寫入log string format
        # self.first_line：log的第一行的格式，通常是欄位名稱
        # self.log_format：log的格式
        self.queue = list()
        self.flush = flush
        self.flush_threshold = flush_threshold
        self.first_line = first_line
        self.log_format = log_format
        self.fname = fname
        with open(fname, 'a') as log_file:
            if self.first_line:
                log_file.write(self.first_line + '\n')
        
    # 透過mode來切換log格式，預設為format，或者個人也可以手動決定log格式
    # 所有log後面皆有後綴字串postfix可以掛上
    def log(self, mode='format', args = tuple(), in_log='', postfix=''):
        if mode == 'format':
            log_str = self.log_format.format(*args) + ' ' + postfix
        elif mode == 'manual':
            log_str = in_log + ' ' + postfix
                
        self.queue.append(log_str)
        
        if len(self.queue) >= self.flush_threshold:
            log_str = '\n'.join(self.queue)
            self.log_flush()
            
    # 要強制寫入log檔時呼叫self.log_flush()，通常是靠self.flush是否為True來判斷要不要呼叫self.log_flush()
    # 執行後會將self.flush設成False
    def log_flush(self):
        if self.queue:
            log_str = '\n'.join(self.queue)
            with open(self.fname, 'a') as log_file:
                log_file.write(log_str + '\n')
            
            self.flush = False
            
            del self.queue
            self.queue = list()    

In [6]:
# In[5]:

from collections import defaultdict
from datetime import datetime
import copy
import pandas as pd
import json
import pickle



# 爬蟲主體
# 查詢動作會有2階段
# phase 1：first_connection()會將查詢關鍵字丟進去，選好搜尋的類別，然後開始搜尋，搜尋結果有時會有多頁，例如搜尋「公司」這個字串，會有100多萬筆
# phase 2：second_connection()針對phase 1的結果，去點「詳細資料」並解析table中不同欄位的資料
class cmpyinfo_crawler:
    #proxies = [{'http':'http://proxy.hinet.net:80'}, {'http':'172.103.3.156:53281'}]
    # 根據「查詢結果」的「詳細資料」頁面，表格左上角的名稱來分類，有以下幾類Cmpy, CmpyCnRp, CmpyCn, CmpyFrgnRp, CmpyFrgn, Busm, BrBusm, Fact, BrCmpy, Lmtd, BrLmtd
    # 以上這幾類的欄位各有不同
    h3_dict = {
        '公司基本資料':'Cmpy',
        '大陸公司許可報備基本資料':'CmpyCnRp', # 53656300 
        '大陸公司許可基本資料':'CmpyCn',# 53026057
        '外國公司報備基本資料':'CmpyFrgnRp', # 16747659
        '外國公司認許基本資料':'CmpyFrgn', # 24812289
        '商業登記基本資料':'Busm',
        '商業登記基本資料(分支機構)':'BrBusm', # 02284257
        '工廠基本資料':'Fact', 
        '分公司資料':'BrCmpy',
        '有限合夥登記基本資料':'Lmtd',
        '有限合夥登記基本資料(分支機構)':'BrLmtd'
    }
    
    # 查詢動作共兩個：
    # 1.先輸入查詢的字串，解析response後會有objectId
    # 2.再根據objectId的前2碼('HC', 'HB'...)去查詢「詳細資料」，不同種類會連到不同的「詳細資料」網頁查詢
    url2_dict = {
        'HC' : 'http://findbiz.nat.gov.tw/fts/query/QueryCmpyDetail/queryCmpyDetail.do',
        'HB' : 'http://findbiz.nat.gov.tw/fts/query/QueryBusmDetail/queryBusmDetail.do',
        'BB' : 'http://findbiz.nat.gov.tw/fts/query/QueryBusmDetail/queryBusmDetail.do',
        'HF' : 'http://findbiz.nat.gov.tw/fts/query/QueryFactDetail/queryFactDetail.do',
        'BC' : 'http://findbiz.nat.gov.tw/fts/query/QueryBrCmpyDetail/queryBrCmpyDetail.do',
        'HL' : 'http://findbiz.nat.gov.tw/fts/query/QueryLmtdDetail/queryLmtdDetail.do',
        'BL' : 'http://findbiz.nat.gov.tw/fts/query/QueryLmtdDetail/queryLmtdDetail.do'
    }
    
    # objectId前2碼與網站中form_data類別的對應
    cmpy_type_dict = {
        'HC' : 'Cmpy',   # 公司
        'HB' : 'Busm',   # 商業
        'BB' : 'Busm',   # 商業
        'HF' : 'Fact',   # 工廠
        'BC' : 'BrCmpy', # 分公司
        'HL' : 'Lmtd',   # 有限合夥
        'BL' : 'BrLmtd'  # 有限合夥分公司
    }
    # 使用 UserAgent套件，以隨機產生request header的user agent
    ua = UserAgent()
    
    def __init__(self, qryCond='', qryType='', pageStart=1, pageEnd=1, path_phantomjs = '/usr/local/Cellar/phantomjs/2.1.1/bin/phantomjs', logname = 'task.log', sleep_scale='small'):
        
        # self.path_phantomjs ： phantomjs執行檔路徑
        # self.session        ： session
        # self.pageStart      ： 當phase 1的結果為多頁，設定要從哪一頁開始爬，目前系統上限能爬的內容為500頁
        # self.pageNow        ： 針對phase 1的多頁結果爬到第幾頁
        # self.pageEnd        ： phase 1的多頁結果最多爬到第幾頁
        # self.pageItem       ： 預設phase 1的結果每頁至多20項，self.phageItem紀錄現在是第幾項
        # self.qryCond        ： 要查詢的關鍵字
        # self.qryType        ： 搜尋的種類，相當於對搜尋頁面中「公司」、「商業」、「工廠」、「分公司」、「有限合夥」核取方塊打勾        
        self.path_phantomjs = path_phantomjs
        self.session = None
        self.pageStart = pageStart 
        self.pageNow = pageStart
        self.pageEnd = pageEnd
        self.pageItem = 1 
        self.qryCond = qryCond
        self.qryType = qryType
        
        # 以下是phase 2時要送的form_data
        # 都必須透過解析phase 1的response來得到，特別是透過phase 1得到的objectId前兩碼來填入對應的值
        # 參：self.get_banKey_objectId()
        # self.banNo      ：phase 2要用到的form data，依objectId前兩碼而異
        # self.brBanNo    ：phase 2要用到的form data，依objectId前兩碼而異
        # self.banKey     ：phase 2要用到的form data，依objectId前兩碼而異
        # self.estbId     ：phase 2要用到的form data，依objectId前兩碼而異
        # self.objectId   ：objectId
        # self.objectId64 ：v.1.10之後ojbectId解析出來還會經過base64編碼，要送出去的object也要經過base64編碼才能成功騙過server
        # self.querytype  ：phase 1可能會有多筆結果，針對每一筆的objectId可以取出前兩碼，self.querytype是用來存objectId前兩碼，並針對這兩碼來決定要送出的form data要長怎樣        
        self.banNo = None
        self.brBanNo = None
        self.banKey = None
        self.estbId = None
        self.objectId = None
        self.objectId64 = None
        self.querytype = None
        
        # self.cookie        ：後來沒用到
        # self.oncontextmenu ：phase 1每一頁若有多筆結果，則結果會存在response的oncontextmenu，我們透過這個list來存phase 1某頁的所有查詢結果
        # self.h3_info       ：後來沒用到
        # self.response      ：收到的response，除了拿來parse之外，主要是debug用
        # self.h3_text       ：phase 2時，詳細資料內會有某些特定的文字（即self.h3_dict的key），依照不同的文字可以分成幾大類
        # self.h3_query      ：對應self.h3_dict[self.h3_text]得到的value，主要是內部做parsing時會按照self.h3_query當作規則
        # self.user_agent    ：header的user_agent
        self.cookie = None
        self.oncontextmenu = list()
        self.h3_info = defaultdict(set)
        self.response = None
        self.h3_text = None
        self.h3_query = None
        self.user_agent = None

        
        # self.sta            ：開始執行時間
        # self.end            ：結束執行時間
        # self.this_round_sta ：phase 1的多頁結果，每一頁起始的時間
        # self.this_round_end ：phase 1的多頁結果，每一夜的結束時間，跟self.this_round_sta相減可以得到處理時間
        # self.pooling        ：單位為秒，self.proxy_monitor()會根據這個值決定何時要換proxy，設為<0的值則不會換proxy
        self.sta = time.time()
        self.end = time.time()
        self.this_round_sta = time.time()
        self.this_round_end = time.time()
        self.pooling = time.time()
        
        
        # self.proxypool    ：proxypool物件
        # self.proxy_ratio  ：proxy每N筆切回None(用自己ip)
        # self.proxy_i      ：後來沒用到
        # self.proxy        ：現在拿到的proxy
        # self.proxy_update ：換了proxy後會設為True，用來判斷做不同處理(例如session要重開)
        # self.proxy_tick   ：self.proxy_monitor()多久換一次proxy
        self.proxypool = proxypool.proxypool(path_phantomjs=path_phantomjs)
        self.proxy_ratio = 5 # 5 proxies vs 1 None
        self.proxy_i = 0
        self.proxy = None
        self.proxy_update = True
        self.proxy_tick = 1800

        # self.totalPage        ：phase 1的結果會有「共n筆、共m頁」的字樣，self.totalPage用來記錄phase 1的結果有幾頁
        # self.totalCount       ：phase 1的結果會有「共n筆、共m頁」的字樣，self.totalPage用來記錄phase 1的結果有幾筆
        # self.flush_threshold  ：設定蒐集到結果每幾筆要寫出到檔案中，現在設定20筆，就是phase 1結果每一頁的最多筆數
        # self.total_json_name  ：寫出的juson檔名
        # self.search_error_cnt ：正常來說有時搜尋的統編查不到資料，但也有可能是proxy問題，設定一直出現search_error幾次才真的跳過
        self.totalPage = 1
        self.totalCount = 1
        self.flush_threshold = 20
        self.total_json_name = "all_json_out"
        self.search_error_cnt = 0
        
        
        # 印出現在的時間
        def tasklog_timestamp():
            d = datetime.now()
            d.year,d.month,d.day,d.hour,d.minute,d.second
            nowstr = "{0:0>4d}{1:0>2d}{2:0>2d}_{3:0>2d}{4:0>2d}{5:0>2d}".format(d.year,d.month,d.day,d.hour,d.minute,d.second)
            return nowstr   
            
        # self.timestamp        ：用tasklog_timestamp()得到的現在時間的格式化字串，紀錄tasklog用
        # self.tasklog          ：back_log物件，
        # self.parser           ：每一種種類都產生一個對應的parser_cmpy_type parser
        self.timestamp = tasklog_timestamp()
        self.tasklog = back_log(flush=False, 
                                flush_threshold=100, 
                                first_line='{0: >19s}, {1: >19s}, {2: >6s}, {3: >6s}'.format('task execution time', 'this round time', 'page', 'item'), 
                                log_format= '{0: >19s}, {1: >19s}, {2: >6d}, {3: >6d}', 
                                fname = '{}@{: >17s}.task.log'.format(logname, str(self.timestamp)))
        self.parser = {'Cmpy':parser_cmpy_type('Cmpy', self.tasklog), 
                       'CmpyFrgnRp':parser_cmpy_type('CmpyFrgnRp', self.tasklog), 
                       'CmpyFrgn':parser_cmpy_type('CmpyFrgn', self.tasklog), 
                       'CmpyCnRp':parser_cmpy_type('CmpyCnRp', self.tasklog), 
                       'CmpyCn':parser_cmpy_type('CmpyCn', self.tasklog), 
                       'BrCmpy':parser_cmpy_type('BrCmpy', self.tasklog), 
                       'Busm':parser_cmpy_type('Busm', self.tasklog), 
                       'BrBusm':parser_cmpy_type('BrBusm', self.tasklog), 
                       'Fact':parser_cmpy_type('Fact', self.tasklog),
                       'Lmtd':parser_cmpy_type('Lmtd', self.tasklog), 
                       'BrLmtd':parser_cmpy_type('BrLmtd', self.tasklog)}
        
        self.tr = dict()
        self.trlog = dict()
        self.objidlog = dict()
        
        self.results = defaultdict(list)        
        self.exception_happened = False
        self.sleep_scale = sleep_scale 
        
        
        
        
    # 印出比較重要的情報，主要是debug用
    def __str__(self):
        attr = {"session":self.session, "qryCond":self.qryCond, "banNo":self.banNo, "brBanNo":self.brBanNo, "banKey":self.banKey, "estbId":self.estbId, "objectId":self.objectId, "querytype":self.querytype, "h3_text":self.h3_text, "h3_query":self.h3_query}        
        return str(attr) + " " + self.exectime('task execution time: ')

    # 將秒數轉換成可讀的時間格式字串
    def timestr(self, t):
        d, r = divmod(t, 86400)
        h, r = divmod(r, 3600)
        m, s = divmod(r, 60)
        return "{: >4d} days {: >2d}:{: >2d}:{: >2d}".format(int(d), int(h), int(m), int(s))
    
    # 執行到目前的時間
    def exectime(self, prefix = ''):
        # task execution time
        self.end = time.time()
        return prefix + self.timestr(self.end - self.sta)
        
    # 爬一頁(最多20筆)的起始時間
    def this_round_time_start(self):
        self.this_round_sta = time.time()
    
    # 爬完一頁(最多20筆)的時間
    def this_round_exectime(self, prefix = ''):
        # page execution time
        self.this_round_end = time.time()
        return prefix + self.timestr(self.this_round_end - self.this_round_sta)

    # 從phase 1得到的objectId前兩碼來填入phase 2要用到的form_data
    # phase 1 response中，每一頁的每一個objectId藏在phase 1 response的(html碼)每一筆的「詳細資料」按鈕
    # 會對應到：<span class="moreLinkMouseOut" onmouseover="hoverMoreLink(this);" onmouseout="unHoverMoreLink(this);" 
    # onclick="javascript:qryDetail('HC53026057', false)" oncontextmenu="javascript:qryDetail('HC53026057', true);
    # return false;">&nbsp;詳細資料&nbsp;</span>
    # attri：<span>元素的oncontextmenu
    def get_banKey_objectId(self, attri):
        # v.1.10版之後，取得的objectId是base64加密過，還要先透過objectId_base64才能填入phase 2的form_data
        def objectId_base64(objectId):
            import base64
            b_objectId = str.encode(objectId) # str to byte str
            encoded_b_objectId = base64.b64encode(b_objectId)
            return encoded_b_objectId.decode('utf8')
            
        # 將attri做字串替換以取得objectId
        self.objectId = (attri.replace("javascript:qryDetail('","")).replace("', true);return false;","")
        self.objectId64 = objectId_base64(self.objectId)
        
        # 取出objectId前2碼
        self.querytype = self.objectId[0:2]
        
        # 取出不objectId前2碼，發出Exception
        try:
            if self.querytype not in cmpyinfo_crawler.url2_dict:
                raise CmpyinfoCrawlerError('QueryTypeError', self.sta)
        except CmpyinfoCrawlerError as ccerr:
            print(ccerr)
            return       
                  

        # 按照objectId前2碼來決定如何填入header
        if self.querytype == 'HC': # 公司
            self.banNo = self.objectId.replace('HC', '')
            self.brBanNo = ''
            self.banKey = ''
            self.estbId = ''
        elif self.querytype == 'BC': # 分公司
            self.banNo = ''
            self.brBanNo = self.objectId.replace('BC', '')
            self.banKey = ''
            self.estbId = ''
        elif self.querytype == 'HB': # 商號
            self.banNo = self.objectId[2:2+8]
            self.brBanNo = ''
            self.banKey = self.objectId[2+8:]
            self.estbId = ''
        elif self.querytype == 'BB': # 商號分公司
            self.banNo = self.objectId[2:2+8]
            self.brBanNo = ''
            self.banKey = self.objectId[2+8:]
            self.estbId = ''            
        elif self.querytype == 'HF': # 工廠
            self.banNo = ''
            self.brBanNo = ''
            self.banKey = ''
            self.estbId = self.objectId.replace('HF', '')
        elif self.querytype == 'HL': # 有限合夥
            self.banNo = self.objectId.replace('HL', '')
            self.brBanNo = ''
            self.banKey = ''
            self.estbId = ''
        elif self.querytype == 'BL': # 有限合夥分支
            self.banNo = self.objectId.replace('BL', '')
            self.brBanNo = ''
            self.banKey = ''
            self.estbId = ''

    
        #print('banNo:', self.banNo, 'brBanNo:', self.brBanNo, 'banKey:', self.banKey, 'estbId:', self.estbId, 'querytype:', self.querytype)
        
    # phase 1：first_connection()要用的的form_data
    # 當一開始查詢時，送出的form_data是以下格式
    def first_connection_header0(self):
        self.form_data_url1 = dict()
        self.form_data_url1 = {'qryCond':str(self.qryCond),
                               'infoType':'D',
                               'qryType':'',
                               'cmpyType':'',
                               'brCmpyType':'',
                               'busmType':'',
                               'factType':'',
                               'lmtdType':'',
                               'isAlive':'all',
                               'sugCont':'',
                               'sugEmail':'',
                               'g-recaptcha-response':''
                              }
        
        for qry in self.qryType:
            if qry == 'cmpyType':
                self.form_data_url1['qryType'] = 'cmpyType'
                self.form_data_url1['cmpyType'] = 'true'
            if qry == 'brCmpyType':
                self.form_data_url1['qryType'] = 'brCmpyType'
                self.form_data_url1['brCmpyType'] = 'true'
            if qry == 'busmType':
                self.form_data_url1['qryType'] = 'busmType'
                self.form_data_url1['busmType'] = 'true'
            if qry == 'factType':
                self.form_data_url1['qryType'] = 'factType'
                self.form_data_url1['factType'] = 'true'            
            if qry == 'lmtdType':
                self.form_data_url1['qryType'] = 'lmtdType'
                self.form_data_url1['lmtdType'] = 'true'            

    # phase 1：first_connection()要用的的form_data
    # 查詢有多頁，想要看第2頁以後的內容時，送出的form_data是以下格式
    def first_connection_header1(self, currentPage):
        self.form_data_url1 = dict()
        self.form_data_url1 = {'pagingModel.totalCount':str(self.totalCount),
                               'pagingModel.currentPage':str(currentPage),
                               'pagingModel.totalPage':str(self.totalPage),
                               'model.qryCond':str(self.qryCond),
                               'model.isAlive':'all',
                               'model.cmpyType':'',
                               'model.brCmpyType':'',
                               'model.busmType':'',
                               'model.factType':'',
                               'model.lmtdType':'',
                               'model.infoType':'D',
                               'model.busiItemSub':'',
                               'model.city':''}        
        
        for qry in self.qryType:
            if qry == 'cmpyType':
                self.form_data_url1['model.cmpyType'] = 'true'
            if qry == 'brCmpyType':
                self.form_data_url1['model.brCmpyType'] = 'true'
            if qry == 'busmType':
                self.form_data_url1['model.busmType'] = 'true'
            if qry == 'factType':
                self.form_data_url1['model.factType'] = 'true'            
            if qry == 'lmtdType':
                self.form_data_url1['model.lmtdType'] = 'true'
                
    # 在phase 1時根據currentPage的數字決定first_connection()要連到第1頁，或第n頁(或自動選則對應的form_data)
    def set_form_data_url1(self, mode, currentPage=1):
        if mode == 0:
            self.first_connection_header0()
        elif mode == 1:
            self.first_connection_header1(currentPage)
    
    # 如果有切換proxy(self.proxy_update為True)則將舊的session關閉，再重開一個新的
    def new_session(self):
        if self.proxy_update:
            if self.session is not None:
                self.session.close()
        
            self.session = requests.Session()
            self.proxy_update = False    
    
    # phase 1的連線流程，主要是填入對應的header, form_data並連到url1，程式碼中有很多exception處理，主要是為了不要中斷程式碼執行
    # 這些exception都是try and error得到的，所以也沒有太高深的理由，遇到就加上去
    # 可以試著先將exception的部分去除，理解整體的流程
    # first_connection()如果正常結束，就return True, 有exception或是搜尋失敗就會return False
    def first_connection(self):
        # phase 1是將request送到url1
        url1 = 'http://findbiz.nat.gov.tw/fts/query/QueryList/queryList.do'
        
        # phase 1會用到的header, Cookie靠requests.session()幫我們維護，所以不手刻
        request_header1 = {
            'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
            'Accept-Encoding':'gzip, deflate',
            'Accept-Language':'zh-TW,zh;q=0.8,en-US;q=0.6,en;q=0.4',
            'Cache-Control':'max-age=0',
            'Connection':'keep-alive',
            ##'Content-Length':'239',
            'Content-Type':'application/x-www-form-urlencoded',
            ##'Cookie':'qryCond=00114003~type=cmpyType,brCmpyType,busmType,factType,lmtdType,~infoType=D~isAlive=all~; JSESSIONID=6D2FA44A8850268F8A6A9D429D53B548; DWRSESSIONID=*o94X5xiFwhD8b5kRGWQo7XpKTl; JSESSIONID=33352345177332F62F210E99D78BD44A; _ga=GA1.3.315276753.1502943360; _gid=GA1.3.754870603.1502943360; _gat=1',
            'Host':'findbiz.nat.gov.tw',
            'Origin':'http://findbiz.nat.gov.tw',
            'Referer':'http://findbiz.nat.gov.tw/fts/query/QueryBar/queryInit.do',
            'Upgrade-Insecure-Requests':'1',
            #'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36'   
        }
        #if get_new_session:
        
        # proxy有換時，就要換新session
        if self.proxy_update:
            # 產生新的 User-Agent
            request_header1['User-Agent'] = cmpyinfo_crawler.ua.random
            # 新的session 會換瀏覽器，並且拿新的session
            # self.new_session()會將proxy_update設為False，所以要放後面
            self.new_session()
           
        try:
            self.response = self.session.post(url1, headers=request_header1, data=self.form_data_url1, proxies=self.proxy)
            
        except requests.exceptions.ProxyError as err:
            self.tasklog.log(mode='manual', in_log = "Exception requests.ProxyError @ first_connection(), remove proxy")
            self.change_proxy()
            #self.response = self.session.post(url1, headers=request_header1, data=self.form_data_url1, proxies=self.proxy)
            return False            
        except Exception as err:
            self.exception_happened = True
            self.tasklog.log(mode='manual', in_log = "Exception @ first_connection()")
            print(err.__doc__)
            self.tasklog.log(mode='manual', in_log = err.__doc__)
            self.tasklog.log_flush()
            self.change_proxy()
            return False
        
        
        try:
            if self.response.status_code != 200:
                raise CmpyinfoCrawlerError('Response200Error', self.sta)
        except CmpyinfoCrawlerError as ccerr:
            self.exception_happened = True
            self.tasklog.log(mode='manual', in_log = "Exception @ first_connection()")
            print(ccerr)
            self.tasklog.log(mode='manual', in_log = str(ccerr))
            self.tasklog.log_flush()
            self.change_proxy()
            return False
        except Exception as err:
            self.exception_happened = True
            self.tasklog.log(mode='manual', in_log = "Exception @ first_connection()")
            print(err.__doc__)
            self.tasklog.log(mode='manual', in_log = err.__doc__)
            self.tasklog.log_flush()
            self.change_proxy()
            return False

       
        try:
            # 對response取出DOM tree以parse出特定元素
            selector = etree.HTML(self.response.content)
        except Exception as err:
            self.exception_happened = True
            self.tasklog.log(mode='manual', in_log = "Exception from etree @ first_connection()")
            print(err.__doc__)
            self.tasklog.log(mode='manual', in_log = err.__doc__)
            self.tasklog.log_flush()
            self.change_proxy()
            return False
       
        
        try:
            # reCaptcha 測試
            # 如果遇到reCpatcha，response會產生元素"g-recaptcha"，所以判斷有沒有這個元素就可以判斷是否撞到reCpatcha
            recaptcha = selector.xpath('//div[@class="g-recaptcha"]')
            if recaptcha:
                raise CmpyinfoCrawlerError('reCaptchaError', self.sta)
        except CmpyinfoCrawlerError as ccerr:
            self.exception_happened = True
            self.tasklog.log(mode='manual', in_log = "Exception @ first_connection()")
            print(ccerr)
            self.tasklog.log(mode='manual', in_log = str(ccerr))
            self.tasklog.log_flush()
            self.change_proxy()
            return False
        except Exception as err:
            self.exception_happened = True
            self.tasklog.log(mode='manual', in_log = "Exception from recaptcha@ first_connection()")
            print(err.__doc__)
            self.tasklog.log(mode='manual', in_log = err.__doc__)
            self.tasklog.log_flush()
            self.change_proxy()
            return False

        
        # 1頁會有筆資料(最多20筆)，要抓出每筆的oncontextmenu，裡面有objectId
        # self.oncontextmenu這個list可以存每頁的objectId,最多20筆,這是phase 2要用的
        hrefs = selector.xpath('//*[@id="vParagraph"]/div[@class="panel panel-default"]/div[2]/span')
        del self.oncontextmenu
        self.oncontextmenu = list()
        for h in hrefs:
            if 'oncontextmenu' in h.attrib and h.attrib['oncontextmenu'] not in self.oncontextmenu:
                self.oncontextmenu.append(h.attrib['oncontextmenu'])


        try:
            # 如果self.oncontextmenu是空的，表示搜尋沒有結果
            if not self.oncontextmenu:
                raise CmpyinfoCrawlerError('searchFailError', self.sta)
            else:
                return True
        except CmpyinfoCrawlerError as ccerr:
            self.search_error_cnt = self.search_error_cnt + 1
            self.exception_happened = True
            self.tasklog.log(mode='manual', in_log = "Exception @ first_connection()")
            print(ccerr)
            self.tasklog.log(mode='manual', in_log = str(ccerr))
            self.tasklog.log_flush()
            self.change_proxy()
            return False                
        except Exception as err:
            self.exception_happened = True
            self.tasklog.log(mode='manual', in_log = "Exception from oncontextmenu@ first_connection()")
            print(err.__doc__)
            self.tasklog.log(mode='manual', in_log = err.__doc__)
            self.tasklog.log_flush()
            self.change_proxy()
            return False

               
    # phase 1取出objectId(存在self.oncontextmenu)後，透過get_banKey_objectId()產生正確的資訊填入phase 2需要的form_data
    def second_connection(self):
        # phase 2是將request送到url2
        url2 = cmpyinfo_crawler.url2_dict[self.querytype]
        
        # phase 2會用到的form_data,這些內容要先執行get_banKey_objectId()
        form_data_url2 = {
            'banNo':str(self.banNo),
            'brBanNo':str(self.brBanNo),
            'banKey':str(self.banKey),
            'estbId':str(self.estbId),
            'objectId':str(self.objectId64),
            'CPage':'',
            'brCmpyPage':'',
            'eng':'',
        }

        # phase 2會用到的header
        request_header2={
            'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
            'Accept-Encoding':'gzip, deflate',
            'Accept-Language':'zh-TW,zh;q=0.8,en-US;q=0.6,en;q=0.4',
            'Cache-Control':'max-age=0',
            'Connection':'keep-alive',
            ##'Content-Length':'211',
            ##'Content-Type':'application/x-www-form-urlencoded',
            ##'Cookie':'JSESSIONID=3193CF8D61AEC43F3810BF07524701BA; DWRSESSIONID=*o94X5xiFwhD8b5kRGWQo7XpKTl; _gat=1; _ga=GA1.3.315276753.1502943360; _gid=GA1.3.754870603.1502943360',
            'Host':'findbiz.nat.gov.tw',
            'Origin':'http://findbiz.nat.gov.tw',
            'Referer':'http://findbiz.nat.gov.tw/fts/query/QueryList/queryList.do',
            ##'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36'
        }
        
        # 產生新的 User-Agent
        ua = UserAgent()
        request_header2['User-Agent'] = cmpyinfo_crawler.ua.random

        try:
            self.response = self.session.post(url2, headers=request_header2,data=form_data_url2, proxies=self.proxy) # first connection有機會重新設定self.proxy, second connection直接用
        except requests.exceptions.ProxyError as err:
            self.tasklog.log(mode='manual', in_log = "Exception requests.ProxyError @ second_connection(), remove proxy")
            self.change_proxy()
            return False
        except Exception as err:
            self.exception_happened = True
            self.tasklog.log(mode='manual', in_log = "Exception @ second_connection()")
            print(err.__doc__)
            self.tasklog.log(mode='manual', in_log = err.__doc__)
            self.tasklog.log_flush()
            self.change_proxy()
            return False
        
        try:
            if self.response.status_code != 200:
                raise CmpyinfoCrawlerError('Response200Error', self.sta)
        except CmpyinfoCrawlerError as ccerr:
            self.exception_happened = True
            self.tasklog.log(mode='manual', in_log = "Exception @ second_connection()")
            print(ccerr)
            self.tasklog.log(mode='manual', in_log = str(ccerr))
            self.tasklog.log_flush()
            self.change_proxy()
            return False
        
        return True
        
    # phase 2會連到每筆結果更詳細的資訊，由於網頁上只有分幾個大類('Cmpy', 'BrCmpy', 'Busm', 'Fact', 'Lmtd')
    # 但實際點進去phase 2產生的response會看到<h3>元素上是有更細的分類，例如'Cmpy'還會有'公司基本資料', '大陸公司許可報備基本資料', '外國公司報備基本資料',...等不同
    # 由<h3>解析出來的中文字串才是比較精確的類型
    def get_h3(self):
        # 對second_connection()得到的response取出DOM tree以做解析
        selector = etree.HTML(self.response.content)
        h3 = selector.xpath('//div[@id="content"]/div[@class="tab-content"]/div[@class="tab-pane active"]/h3')
        

        try:
            # 如果<h3>沒有內容表示有誤
            if len(h3) == 0:
                raise CmpyinfoCrawlerError('h3ResolveError', self.sta)
        except CmpyinfoCrawlerError as ccerr:
            self.exception_happened = True
            self.tasklog.log(mode='manual', in_log = "Exception @ get_h3(): can't resolve h3 in xpath")
            print(ccerr)
            self.tasklog.log(mode='manual', in_log = str(ccerr))
            self.tasklog.log_flush()
            return False
        
        # 對<h3>的中文字串做處理, 結果的中文字串會存在self.h3_text
        self.h3_text = h3[0].text.encode('latin_1', errors='ignore').decode('utf8', errors='ignore')
        self.h3_text = re.sub(r'\s', r'', self.h3_text)
        print('self.h3_text resolved: ', self.h3_text)
        #print(self.h3_text)
        if self.h3_text in cmpyinfo_crawler.h3_dict:
            self.h3_query = cmpyinfo_crawler.h3_dict[self.h3_text]
            print('self.h3_query resolved: ', self.h3_query)
        
        try:
            # 如果<h3>解析出來的中文字串是不認識的種類，表示有誤，但有可能隨著網頁的版本會有新的種類，若發生錯誤最好把self.h3_text印出來
            if self.h3_text not in cmpyinfo_crawler.h3_dict:
                raise CmpyinfoCrawlerError('h3ResolveError', self.sta)
        except CmpyinfoCrawlerError as ccerr:
            self.exception_happened = True
            self.tasklog.log(mode='manual', in_log = "Exception @ get_h3(): new h3 found")
            print(ccerr)
            self.tasklog.log(mode='manual', in_log = str(ccerr))
            self.tasklog.log_flush()
            return False
        
        return True
            

    # 判斷是否要切換proxy
    # 如果self.proxy_tick < 0，就不檢查
    # 如果self.proxy_tick > 0就每隔self.proxy_tick會切換一次proxy
    def proxy_monitor(self):
        if self.proxy_tick < 0:
            return

        self.end = time.time()        
        if self.end - self.pooling > self.proxy_tick:
            self.change_proxy()
            self.pooling = time.time()
    
    # self.proxypool的proxy用完時要重新搜集可用的proxy(複數)到proxypool
    def renew_poroxypool(self):
        self.proxypool.reset_proxy()

        
        try:
            # self.proxypool去group_proxy()搜集proxy(複數),失敗的話會再嘗試一次，兩次都失敗就算是更新失敗
            self.proxypool.group_proxy()
            if len(self.proxypool.proxy_set) == 0:
                raise CmpyinfoCrawlerError('ResolveProxyError', self.sta)
        except CmpyinfoCrawlerError as cerr:
            self.exception_happened = True
            self.tasklog.log(mode='manual', in_log = "Can not resolve proxy from proxypool.group _proxy(), retry proxypool.group_proxy()")
            print(ccerr)
            self.tasklog.log(mode='manual', in_log = str(ccerr))
            self.tasklog.log_flush()
            
            try:
                # self.proxypool去group_proxy()搜集proxy(複數),失敗的話會嘗試第二次，兩次都失敗就算是更新失敗
                self.proxypool.group_proxy()
                if len(self.proxypool.proxy_set) == 0:
                    raise CmpyinfoCrawlerError('ResolveProxyError', self.sta)                
            except:
                self.exception_happened = True
                self.tasklog.log(mode='manual', in_log = "Can not resolve proxy from proxypool.group_proxy(), you can only use local ip")
                print(ccerr)
                self.tasklog.log(mode='manual', in_log = str(ccerr))
                self.tasklog.log_flush()
                return False
            
                
        proxy_str = 'resolved ' + str(len(self.proxypool.proxy_set)) + ' proxies'
        print(proxy_str)
        self.tasklog.log(mode='manual', in_log = proxy_str)
        return True

    # 把self.proxy設成參數p
    def set_proxy(self, p):
        self.proxy = p
        proxy_str = 'proxy change to ' + str(self.proxy) + ' @ ' + self.exectime('task execution time: ')
        print(proxy_str)
        self.tasklog.log(mode='manual', in_log = proxy_str)
        self.proxy_update = True

        
    # 如果proxypool有東西，就按照random_choice_one_proxy_with_none_freq()挑出的proxy設為self.proxy
    # 如果proxypool是空的就 renew_poroxypool()一下
    def change_proxy(self):
        if not self.proxypool.proxy_set:
            self.renew_poroxypool()
        
        p = self.proxypool.random_choice_one_proxy_with_none_freq()
        
        self.set_proxy(p)
        

    # 設定sleep的時間，可以根據scale參數設定不同的檔位，'none', 'mixed', 'mixed_small', 'mixed_medium', 'mixed_large', 'small', 'midium', 'large'
    # 不同檔位讓parse每一筆資料中間有不同時間長度的time.sleep()
    def random_sleep(self, scale = 'small'):
        import random

        #            15%   20%     20%     10% 15%   20%   
        sleeptime = [1,1,1,2,2,2,2,3,3,3,3,4,4,5,5,5,6,6,6,6]
        #            15%               35%                  15%               15%            5%    15%   
        magnitude = [0.75, 0.75, 0.75, 1, 1, 1, 1, 1, 1, 1, 1.25, 1.25, 1.25, 1.5, 1.5, 1.5, 1.75, 2, 2, 2]
        #            10%20%   30%      40%
        basetime  = [5, 7.5, 7.5, 10, 12.5, 12.5, 3, 3, 3, 3]
        
        if scale == 'none':
            return

        if scale == 'mixed':
            scale = random.choice(['small']+['midium']+['large'])
        if scale == 'mixed_small':
            scale = random.choice(['small']*14+['midium']*3+['large']*3)
        if scale == 'mixed_medium':
            scale = random.choice(['small']*3+['midium']*14+['large']*3)
        if scale == 'mixed_large':
            scale = random.choice(['small']*3+['midium']*3+['large']*14)
        
        m = random.choice(magnitude)
        b = random.choice(basetime)        
        s = random.choice(sleeptime)
        if scale == 'small':
            s *= 1
            b *= 0.25
            m *= 0.75
        if scale == 'midium':
            s *= 6
            b *= 0.75
            m *= 0.75
        if scale == 'large':
            s *= 12
            b *= 1
            m *= 1 
            
        time.sleep(b+m*s)
            
    # 把self.response.content印出來，方便debug
    def print_html(self):
        print(self.response.content.decode('utf8'))
        
    # 根據first_connection()的結果解析出這個搜尋有「幾筆」(存在self.totalcount), 共「幾頁」(存在self.totalPage)
    def resolve_page(self):
        selector = etree.HTML(self.response.content)
        text = selector.xpath('//*[@id="queryListForm"]/div[@class="main"]/div[@class="container padding_top"]/div[@class="row"]/div[@class="col-lg-12"]/div/div/div/text()')

        text = text[1].encode('latin_1', errors='ignore').decode('utf8', errors='ignore')
        print(text)
        self.tasklog.log(mode='manual', in_log = 'task resolved: '+ text)
        text = re.sub(r'[\s共筆分頁,]', r'', text)
        print(text)
        self.totalCount, self.totalPage = [int(t) for t in text.split('、')]
           
    # 如果頁面是「'很抱歉，您所存取的網頁系統暫時無法回應。'」return True, 否則return False
    def second_page_not_found(self):
        selector = etree.HTML(crawler.response.content)
        texts = selector.xpath('//table/tr/td/table/tr/td/descendant::*/text()')
        
        clean_texts = list()
        for t in texts:
            #t = t.encode('latin_1', errors='ignore').decode('utf8', errors='ignore')
            t = re.sub(r'\s', r'', t)
            if t:
                clean_texts.append(t)
        else:
            if '很抱歉，您所存取的網頁系統暫時無法回應。' in clean_texts and '回上一頁' in clean_texts and '回首頁' in clean_texts:
                return True
            else:
                return False
      
    # 將parse出來的東西寫入檔案，目前是將爬到的內容轉成json並以append方式寫入 self.total_json_name+'_json.json'
    def output_files(self, itemStart, itemEnd):
        d = datetime.now()
        timestamp = "{0:0>4d}{1:0>2d}{2:0>2d}_{3:0>2d}{4:0>2d}{5:0>2d}".format(d.year,d.month,d.day,d.hour,d.minute,d.second)
        fname = '{0:　>5s}@{1: >10s}-{2: >17s}'.format(self.qryCond, str(self.qryType[0]),timestamp)        
        #fname = '{0:　>5s}@{1: >10s} [{2: >6d}-{3: >6d}][{4: >6d}-{5: >6d}]-{6: >17s}'.format(self.qryCond, str(self.qryType[0]), self.pageStart, self.pageEnd, itemStart, itemEnd,timestamp)        
        for key in self.results:
            if self.results[key]:
                #j = json.dumps(self.results[key], ensure_ascii=False)
                j = self.results[key]
                #with open(fname+'_'+key+'_json.json', 'w') as jout:
                #    json.dump(j, jout, ensure_ascii=False)
                with open(self.total_json_name+'_json.json', 'a') as tjout:
                    json.dump(j, tjout, ensure_ascii=False)
                    tjout.write(',\n')
                
                #with open(fname+'_'+key+'_json.pkl', 'wb') as jpklout:  
                #    pickle.dump(j, jpklout)
                    
                #df = pd.DataFrame(self.results[key])
                #with open(fname+'_'+key+'df.csv', 'w') as dfcsvout:
                #    df.to_csv(dfcsvout, index=False)
                #with open(fname+'_'+key+'df.pkl', 'wb') as dfpklout:
                #    pickle.dump(j, dfpklout)
                    
                #with open(fname+'_'+key+'content.html', 'w') as contentout:
                #    contentout.write(self.response.content.decode('utf8'))
                #    #pickle.dump(self.response.content.decode('utf8'), contentout)
                  
    # 紀錄parse某頁開始的時間
    def set_pageStart(self, pageStart):
        self.pageStart = pageStart

    # 紀錄parse某頁結束的時間
    def set_pageEnd(self, pageEnd):
        self.pageEnd = pageEnd


    # 對某個搜尋的結果做逐筆parsing，parse到的結果存入self.result
    def parse_and_gen_schema(self, pageStart=1, pageEnd=1): 
        self.set_pageStart(pageStart)
        self.set_pageEnd(pageEnd)
        
        # 一些counter方便紀錄執行過程
        item_count = 0
        item_last = 0
        item_count_flush = 0
        
        # 針對搜尋結果的每個page去搜尋每頁中的每1筆
        for self.pageNow in range(self.pageStart, self.pageEnd+1):
            self.this_round_time_start()
            
            # 如果現在parse的是第1頁或第n頁就設不同form_data
            if self.pageNow > 1:
                self.set_form_data_url1(mode = 1, currentPage = self.pageNow)
            else:
                self.set_form_data_url1(mode = 0)
              
            # 在first_connection()刷完一頁(最多20筆)，檢查一次是否超過self.proxy_tick，若是就關掉舊的session，並切到新proxy
            self.proxy_monitor()
            
            # self.first_connection() 出問題時重試10次，連續10次first_connection()都return False(即有exception)
            # 就放棄這一頁，改下一頁
            retry = 0
            while not self.first_connection():
                if retry > 10:
                    with open('skip_query.log', 'a') as skpf:
                        skpf.write(self.qryCond+str(self.qryType)+'\n')
                    continue
                
                retry += 1
                self.tasklog.log(mode='manual', in_log = 'first_connection() @ page{} failed, retry {} times'.format(self.pageNow, retry))
                # 每次retry都time.sleep() 2秒
                time.sleep(2)
                
            # 如果first_connection()成功，對該頁(最多20筆)的每筆資料parse
            for self.pageItem, o in enumerate(self.oncontextmenu, 1):
                self.get_banKey_objectId(o)
                
                # self.second_connection() 出問題時重試10次，連續10次second_connection()都return False(即有exception)
                # 就放棄這一筆，改下一筆
                retry = 0
                while not self.second_connection():
                    if retry > 10:
                        continue
                    retry += 1
                    self.tasklog.log(mode='manual', in_log = 'second_connection() @ page{}  item {} failed, retry {} times'.format(self.pageNow, self.pageItem, retry))
                    # 每次retry都time.sleep() 10秒
                    time.sleep(10)

                
                # 如果self.second_connection()網頁不存在，跳過
                if self.second_page_not_found():                    
                    self.tasklog.log(mode='format', args=(self.exectime(), self.this_round_exectime(), self.pageNow, self.pageItem), postfix = 'second page not found')                    
                    continue
                
                
                # 取出self.second_connection()結果<h3>的文字
                self.get_h3()

                # 針對<h3>的文字結果map到適當的parser_cmpy_type並以變數parser暫存
                # parser_cmpy_type會將每一筆資料按照網頁的schema以{欄位:內容}的形式做成dict，存在parser.data_schema
                parser = self.parser[self.h3_query]
                # 將parser.data_schema清空
                parser.init_data_schema()
                # 開始parse second_connection()的內容
                # 並把form_data跟它對應到第幾頁、第幾筆包起來傳給parser_cmpy_type做log
                parser.parser(self.response, 
                              {'banNo':str(self.banNo),
                               'brBanNo':str(self.brBanNo),
                               'banKey':str(self.banKey),
                               'estbId':str(self.estbId),
                               'objectId':str(self.objectId)},
                              {'page':str(self.pageNow),
                               'item':str(self.pageItem)})
                
                
            

                self.exectime(self.h3_query)
                print(self.h3_query, parser.data_schema)             

                
                # 額外增加這些欄位，主要是為了要掌握進度用，用來辨識這一筆資料是哪個關鍵字，第幾頁，第幾筆
                parser.data_schema['qryCond'] = self.qryCond
                parser.data_schema['h3_query'] = self.h3_query
                parser.data_schema['pageNow'] = self.pageNow
                parser.data_schema['pageItem'] = self.pageItem
                
                #將結果存到self.results
                self.results[self.h3_query].append( copy.deepcopy(parser.data_schema) )
                self.tasklog.log(mode='format', args=(self.exectime(), self.this_round_exectime(), self.pageNow, self.pageItem), postfix = 'done') 
                self.random_sleep(scale = self.sleep_scale)
                item_count += 1
                item_count_flush += 1
            else:
                # 將結果寫入json檔，如果self.flush_threshold設為20，就是每頁抓完20筆才寫入一次
                self.tasklog.log(mode='format', args=(self.exectime(), self.this_round_exectime(), self.pageNow, self.pageItem), postfix = 'page finished, total {: >6d} items'.format(item_count))
                print('==================================================')
                print('page', self.pageNow, 'total', item_count , ' time', self.exectime(), 'round time', self.this_round_exectime())
                print('==================================================')
                if item_count_flush >= self.flush_threshold:
                    item_count_flush = 0
                    
                    self.output_files(item_last, item_count)
                    print('flush results to pandas pages:{} items:{}'.format(self.pageNow, item_count))
                    self.tasklog.log(mode='manual', in_log = 'flush results to pandas pages:{} items:{}'.format(self.pageNow, item_count))
                    
                    del self.results
                    self.results = defaultdict(list)
                    item_last = item_count
                    #return
                #self.results.append( (self.querytype, parser.data_schema) )
                self.random_sleep(scale = self.sleep_scale)
        else:
            # 最後一頁不論有幾筆，都寫入json檔
            self.output_files(item_last, item_count)
            print('flush results to pandas pages:{} items:{}'.format(self.pageNow, item_count))
            self.tasklog.log(mode='manual', in_log = 'flush results to pandas pages:{} items:{}'.format(self.pageNow, item_count))

            print('end of parse_and_gen_schema() pages:{} items:{}'.format(self.pageNow, item_count))
            self.tasklog.log(mode='manual', in_log = 'end of parse_and_gen_schema() pages:{} items:{}'.format(self.pageNow, item_count))
            self.tasklog.log_flush()
            
            del self.results
            self.results = defaultdict(list)
            item_last = item_count

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.7
Traceback (most recent call last):
  File "/Users/bonzo/Workspace/python/python3/anaconda3/envs/py35/lib/python3.5/site-packages/fake_useragent/utils.py", line 67, in get
    context=context,
  File "/Users/bonzo/Workspace/python/python3/anaconda3/envs/py35/lib/python3.5/urllib/request.py", line 163, in urlopen
    return opener.open(url, data, timeout)
  File "/Users/bonzo/Workspace/python/python3/anaconda3/envs/py35/lib/python3.5/urllib/request.py", line 466, in open
    response = self._open(req, data)
  File "/Users/bonzo/Workspace/python/python3/anaconda3/envs/py35/lib/python3.5/urllib/request.py", line 484, in _open
    '_open', req)
  File "/Users/bonzo/Workspace/python/python3/anaconda3/envs/py35/lib/python3.5/urllib/request.py", line 444, in _call_chain
    result = func(*args)
  File "/Users/bonzo/Workspace/python/python3/anaconda3/envs/py35/lib/python3.5/urllib/

In [7]:
# In[6]:

import copy

# parser主體，跟據「每一筆」的<h3>分類產生不同規則的parser
class parser_cmpy_type:
    
    # 不同<h3>分類在網頁中呈現的schema(table)會不同，所以要parse的欄位名稱就不同，要處理的方式也不同
    # 欄位名稱都會定義在col_Cmpy, col_CmpyFrgnRp...的list中，
    # 欄位名稱對應的parse規則定義在rule_Cmpy, rule_CmpyfrgnRp...的dict中
    
    # 公司基本資料
    col_Cmpy=['統一編號', '公司狀況', '公司名稱', '資本總額(元)', '實收資本額(元)', '代表人姓名', 
              '公司所在地', '登記機關', '核准設立日期', '最後核准變更日期', '停業日期(起)', '停業日期(迄)',
              '所營事業資料' ]
    rule_Cmpy={
        '統一編號':'rule0',
        '公司狀況':'rule0',
        '公司名稱':'rule0',
        '資本總額(元)':'rule0',
        '實收資本額(元)':'rule0',
        '代表人姓名':'rule0',
        '公司所在地':'rule0',
        '登記機關':'rule0',
        '核准設立日期':'rule0',
        '最後核准變更日期':'rule0',
        '停業日期(起)':'rule0',
        '停業日期(迄)':'rule0',
        '所營事業資料':'rule5'
    }
    # 公司基本資料之外，還會有不同頁的資料，諸如公司基本資料，董監事資料，經理人資料，分公司資料，多元查詢
    # 但這邊完全沒開發
    col_Cmpy_Shareholder=['序號','職稱','姓名','所代表法人','持有股份數']
    col_Cmpy_Mgr=[]
    col_Cmpy_BrCmpy=['序號','統一編號','分公司名稱','公司狀況','分公司核准設立日期','最後核准變更日期']
    
    # 外國公司報備基本資料
    col_CmpyFrgnRp=['統一編號', '公司狀況', '公司名稱', '訴訟及非訴訟代理人姓名', '辦事處所在地', '登記機關',
                    '核准設立日期', '最後核准變更日期', '代表人在中華民國境內所為的法律行為']    
    rule_CmpyFrgnRp={
        '統一編號':'rule0',
        '公司狀況':'rule0',
        '公司名稱':'rule0',
        '訴訟及非訴訟代理人姓名':'rule0',
        '辦事處所在地':'rule0',
        '登記機關':'rule0',
        '核准設立日期':'rule0',# 會有空值 <td></td>
        '最後核准變更日期':'rule0',# 會有空值 <td></td>
        '代表人在中華民國境內所為的法律行為':'rule0'
        #'所營事業資料':'rule5'
    }

    # 外國公司認許基本資料
    col_CmpyFrgn=['統一編號', '公司狀況', '公司名稱', '在中華民國境內營運資金', '訴訟及非訴訟代理人姓名', '分公司所在地',
                  #'公司所在地', 
                  '登記機關', '核准設立日期', '最後核准變更日期', '停業日期(起)', '停業日期(迄)', '所營事業資料']
    rule_CmpyFrgn={
        '統一編號':'rule0',
        '公司狀況':'rule0',# 處理跨<br>串字，<br>後面可能是空字串
        '公司名稱':'rule0',
        '在中華民國境內營運資金':'rule0',
        '訴訟及非訴訟代理人姓名':'rule0',
        '分公司所在地':'rule0',
        #'公司所在地':'rule0',
        '登記機關':'rule0',
        '核准設立日期':'rule0',
        '最後核准變更日期':'rule0', # <td></td>
        '停業日期(起)':'rule0',
        '停業日期(迄)':'rule0',
        '所營事業資料':'rule5'
    }
    col_CmpyFrgn_BrCmpy=['序號','統一編號','分公司名稱','公司狀況','分公司核准設立日期','最後核准變更日期']


    
    # 大陸公司許可報備基本資料
    col_CmpyCnRp=['統一編號', '公司狀況', '股權狀況', '公司名稱', '訴訟及非訴訟代理人姓名', '辦事處所在地',
                  '登記機關', '核准設立日期', '最後核准變更日期', '代表人在台灣地區業務活動範圍']    
    rule_CmpyCnRp={
        '統一編號':'rule0',
        '公司狀況':'rule0',
        '股權狀況':'rule0',
        '公司名稱':'rule0',# 有<br>串起來
        '訴訟及非訴訟代理人姓名':'rule0',
        '辦事處所在地':'rule0',
        '登記機關':'rule0',
        '核准設立日期':'rule0',
        '最後核准變更日期':'rule0',
        '代表人在台灣地區業務活動範圍':'rule0'
        #'所營事業資料':'rule5'
    }
    
    col_CmpyCnRp_BrCmpy=['序號','統一編號','分公司名稱','公司狀況','分公司核准設立日期','最後核准變更日期']



    # 大陸公司許可基本資料
    col_CmpyCn=['統一編號', '公司狀況', '股權狀況', '公司名稱', '在台灣地區營業所用', '訴訟及非訴訟代理人姓名', 
                '分公司所在地', #'公司所在地', 
                '登記機關', '核准設立日期', '最後核准變更日期', 
                #'停業日期(起)', '停業日期(迄)', 
                '所營事業資料']
    rule_CmpyCn={
        '統一編號':'rule0',
        '公司狀況':'rule0',
        '股權狀況':'rule0',
        '公司名稱':'rule0',# 有<br>串起來
        '在台灣地區營業所用':'rule0',
        '訴訟及非訴訟代理人姓名':'rule0',
        '分公司所在地':'rule0',
        #'公司所在地':'null',
        '登記機關':'rule0',
        '核准設立日期':'rule0',
        '最後核准變更日期':'rule0',
        #'停業日期(起)':'null',
        #'停業日期(迄)':'null',
        '所營事業資料':'rule5'
    }
    col_CmpyCn_BrCmpy=['序號','統一編號','分公司名稱','公司狀況','分公司核准設立日期','最後核准變更日期']
    
    # 分公司資料
    col_BrCmpy=['分公司統一編號', '分公司狀況', '分公司名稱', '分公司經理姓名', '分公司所在地', '核准設立日期', 
                #'廢止日期', 
                '最後核准變更日期', '總(本)公司統一編號', '總(本)公司名稱']
    rule_BrCmpy={
        '分公司統一編號':'rule0',
        '分公司狀況':'rule0',
        '分公司名稱':'rule0',
        '分公司經理姓名':'rule0',
        '分公司所在地':'rule0',
        '核准設立日期':'rule0',
        #'廢止日期':'rule0',
        '最後核准變更日期':'rule0',
        '總(本)公司統一編號':'rule1', # <br><a></a></br>
        '總(本)公司名稱':'rule1' # <br><a>"雙"<img>"有限公司"</a></br>要處理未識別字有圖的狀況
    }
    
    # 商業登記基本資料
    col_Busm=['登記機關', '商業統一編號', '核准設立日期', '最近異動日期', '商業名稱', '負責人姓名',
              '負責人姓名-出資額(元)', '現況', '資本額(元)', '組織類型', '地址', '營業項目']
    
    
    """
    <tr>
    <td class="txt_td">負責人姓名</td>
        <td>
            <table style="width:100%">
                <tbody><tr>
                    <td width="50%">蔡培火</td>         rule21
                    <td width="50%">出資額(元):0</td>   rule22
                </tr></tbody></table>
        </td>
    </tr>
    """
    
    rule_Busm={
        '登記機關':'rule0',
        '商業統一編號':'rule0', # 可能只有訂閱鈕
        '核准設立日期':'rule0', 
        '最近異動日期':'rule0',
        '商業名稱':'rule0', # <td><a>"雙"<img>"有限公司"</a></td>要處理未識別字有圖的狀況
        '負責人姓名':'rule2',
        '負責人姓名-出資額(元)':'rule2',# 出資額(元):0 
        '合夥人姓名':'rule2',
        '合夥人姓名-出資額(元)':'rule2',
        '現況':'rule0',
        '資本額(元)':'rule0',
        '組織類型':'rule0',
        '地址':'rule0',
        '營業項目':'rule5' # 各種金紙冥紙等有關祭拜用品零售買賣（炮竹除外）@@
    }
    col_Busm_BrBusm=['分支機構統一編號','分支機構名稱','分支機構地址','分支機構經理人姓名'] 
    
    # 商業登記基本資料(分支機構)
    col_BrBusm=['分支機構登記機關', '分支機構統一編號', '核准設立日期', '最近異動日期', '分支機構名稱', '分支機構經理人姓名', 
                '分支機構現況', '分支機構地址', '總機構統一編號', '總(本)商業名稱']
    rule_BrBusm={'分支機構登記機關':'rule0', 
                 '分支機構統一編號':'rule0', 
                 '核准設立日期':'rule0',
                 '最近異動日期':'rule0',
                 '分支機構名稱':'rule0',
                 '分支機構經理人姓名':'rule0',
                 '分支機構現況':'rule0',
                 '分支機構地址':'rule0', 
                 '總機構統一編號':'rule1',
                 '總(本)商業名稱':'rule1'}
    
    # 工廠基本資料
    col_Fact=['登記機關', '工廠名稱', '工廠登記編號', '工廠登記核准日期', '工廠設立許可案號', '工廠設立核准日期', 
              '工廠地址', '工廠負責人姓名', '公司(營利事業)統一編號', '工廠組織型態', '工廠資本額', '工廠登記狀態',
              '最後核准變更日期', '工廠設立許可廢止核准日期', '工廠登記歇業核准日期', '工廠登記廢止核准日期', '工廠登記公告廢止核准日期', '最近一次校正年度', 
              '最近一次校正結果', '依據行政院主計處『中華民國行業標準分類』100年3月第9次修訂', '產業類別', '主要產品']
    rule_Fact={
        '登記機關':'rule0',
        '工廠名稱':'rule0',
        '工廠登記編號':'rule0',
        '工廠登記核准日期':'rule0',
        '工廠設立許可案號':'rule0',
        '工廠設立核准日期':'rule0',
        '工廠地址':'rule0',
        '工廠負責人姓名':'rule0',
        '公司(營利事業)統一編號':'rule0',
        '工廠組織型態':'rule0',
        '工廠資本額':'rule0',
        '工廠登記狀態':'rule0',
        '最後核准變更日期':'rule0',
        '工廠設立許可廢止核准日期':'rule0',
        '工廠登記歇業核准日期':'rule0',
        '工廠登記廢止核准日期':'rule0',
        '工廠登記公告廢止核准日期':'rule0',
        '最近一次校正年度':'rule0',
        '最近一次校正結果':'rule0',
        '依據行政院主計處『中華民國行業標準分類』100年3月第9次修訂':'rule0', # 這個tr是有<br>的
        #'依據行政院主計處『中華民國行業標準分類』':'null',
        '產業類別':'rule5', # 單一項最後面有<br>
        '主要產品':'rule5'
    }
    # 有限合夥登記基本資料
    col_Lmtd=['登記機關', '統一編號', '有限合夥名稱', '所在地', '實收出資額(元)', '核准設立日期', '現況', 
              '存續期間', '最近一次登記狀況核准日期及文號', '代表人姓名', '代表人姓名-出資額(元)', '普通合夥人姓名-責任類型', '普通合夥人姓名',
              '普通合夥人姓名-出資額(元)', '普通合夥人姓名-責任類型', '有限合夥人', '有限合夥人-出資額(元)', '有限合夥人-責任類型', '經理人姓名',
              '約定解散事由', '所營事業項目']
    rule_Lmtd={
        '登記機關':'rule0',
        '統一編號':'rule0',
        '有限合夥名稱':'rule0',
        '所在地':'rule0',
        '實收出資額(元)':'rule0',
        '核准設立日期':'rule0',
        '現況':'rule0',
        '存續期間':'rule0',
        '最近一次登記狀況核准日期及文號':'rule0',# 標題有用<br>隔開
        '代表人姓名':'rule2',
        '代表人姓名-出資額(元)':'rule2',
        '代表人姓名-責任類型':'rule2',
        '普通合夥人姓名':'rule2',
        '普通合夥人姓名-出資額(元)':'rule2',
        '普通合夥人姓名-責任類型':'rule2',
        '有限合夥人':'rule2',
        '有限合夥人-出資額(元)':'rule2',
        '有限合夥人-責任類型':'rule2',
        '經理人姓名':'rule0',
        '約定解散事由':'rule0',
        '所營事業項目':'rule5'
    }
    col_Lmtd_BrBusm=['分支機構統一編號','分支機構名稱','分支機構地址','分支機構經理人姓名']
    
    # 有限合夥登記基本資料(分支機構)
    col_BrLmtd=['登記機關', '統一編號', '分支機構名稱', '所在地', '在中華民國境內營運資金', '核准設立日期', 
                '登記狀況', '最近一次登記狀況核准日期及文號', '在中華民國境內負責人', '分支機構經理人', '所營事業項目', 
                '本機構統一編號', '本機構名稱']
    rule_BrLmtd={
        '登記機關':'rule0',
        '統一編號':'rule0',
        '分支機構名稱':'rule0',
        '所在地':'rule0',
        '在中華民國境內營運資金':'rule0',
        '核准設立日期':'rule0',
        '登記狀況':'rule0',
        '最近一次登記狀況核准日期及文號':'rule0',# 標題有用<br>隔開
        '在中華民國境內負責人':'rule0',
        '分支機構經理人':'rule0',
        '所營事業項目':'rule5',
        '本機構統一編號':'rule0',
        '本機構名稱':'rule0'
    }
    
    # 根據<h3>的類型決定
    col_book = {'Cmpy':col_Cmpy, 
                'CmpyFrgnRp':col_CmpyFrgnRp, 
                'CmpyFrgn':col_CmpyFrgn, 
                'CmpyCnRp':col_CmpyCnRp, 
                'CmpyCn':col_CmpyCn, 
                'BrCmpy':col_BrCmpy, 
                'Busm':col_Busm, 
                'BrBusm':col_BrBusm, 
                'Fact':col_Fact, 
                'Lmtd':col_Lmtd, 
                'BrLmtd':col_BrLmtd}
    
    # 根據<h3>的類型決定有哪些不同頁的資料，未開發
    col_sub_book = {'Cmpy':[col_Cmpy_Shareholder, col_Cmpy_Mgr, col_Cmpy_BrCmpy],
                    'CmpyFrgnRp':[], 
                    'CmpyFrgn':[col_CmpyFrgn_BrCmpy], 
                    'CmpyCnRp':[col_CmpyCnRp_BrCmpy], 
                    'CmpyCn':[col_CmpyCn_BrCmpy], 
                    'BrCmpy':[], 
                    'Busm':[col_Busm_BrBusm], 
                    'BrBusm':[], 
                    'Fact':[], 
                    'Lmtd':[col_Lmtd_BrBusm], 
                    'BrLmtd':col_BrLmtd}
    
    #根據<h3>的類型決定parser的規則
    rule_book = {'Cmpy':rule_Cmpy, 
                 'CmpyFrgnRp':rule_CmpyFrgnRp, 
                 'CmpyFrgn':rule_CmpyFrgn, 
                 'CmpyCnRp':rule_CmpyCnRp, 
                 'CmpyCn':rule_CmpyCn, 
                 'BrCmpy':rule_BrCmpy, 
                 'Busm':rule_Busm, 
                 'BrBusm':rule_BrBusm, 
                 'Fact':rule_Fact, 
                 'Lmtd':rule_Lmtd, 
                 'BrLmtd':rule_BrLmtd}
     
    def __init__(self, h3_query, tasklog):
        self.h3_query = h3_query
        self.tr_rule = parser_cmpy_type.rule_book[self.h3_query]        
        self.data_schema = dict()
        # self.rule0, self.rule1, self.rule2, self.rule5都是針對某種欄位parse的function
        # 以下定義處理哪個rule就呼叫哪個function
        self.td_rule_handler = {
            'rule0'   :self.rule0,
            'rule1'   :self.rule1,
            'rule2'   :self.rule2,
            #'rule3'   :self.special_rule3,
            #'rule4'   :self.special_rule4,
            'rule5'   :self.rule5,
        }
        self.tasklog = tasklog
        
    # parse完的結果存在這，對每筆查詢來說，它的schema的每個欄位會變成{欄位名:字串}的key-value pair, 而整個schema會變成一個dict
    def init_data_schema(self):
        self.data_schema = dict()
    
    # 將文字轉編碼，並去除空白跟'「」'
    def clean_text(self, text):
        text = text.encode('latin_1', errors='ignore').decode('utf8', errors='ignore')
        text = re.sub(r'\s', r'', text)
        text = re.sub(r'[「」]',r'', text)
        return text
        
    # 清除民國日期，改成西元，未使用
    def clean_Minguo_calendar(self, text):
        #if text is None or not text:
        if text is None or not len(text):
            return ""
            
        text_year = text.split('年')[0]
        text_month = text.split('年')[1].split('月')[0]
        text_day = text.split('年')[1].split('月')[1].replace('日','')
        text_year = str(int(text_year) + 1911)
        return "{}-{}-{}".format(text_year, text_month, text_day)
    
    # 將td內的 '<br>'去除
    def td_remove_br(self, td):
        """
        preprocess ./tr/td, remove <br> between text()
        """
        if td is None:
            return ""
        
        tds = td.xpath('./td')
        text = list()
        for t in td.xpath('./text()'):
            t = t.encode('latin_1', errors='ignore').decode('utf8', errors='ignore')
            t = re.sub(r'\s', r'', t)
            if t:
                text.append(t)
        #print(''.join(text))
        
        return ''.join(text)
                        
        
    # 將td內的字串取出，根據sp在輸出之前幾髒串以spｙ握為串接詞，方便後續切分
    # 目前歸納起來會遇到的欄位類型可分成下面註解中的 A cell, B cell, C cell
    def get_td_text(self, td, sp='', rule=0):
        import re
        """
        get td text from type A cell:
        <td class="txt_td">
            "最近一次登記狀況"
            <br>
            "核准日期及文號"
        </td>
        
        get td text from type B cell:
        <td>
            "
                    大陸商中國銀行股份有限公司(在臺灣地區公司名稱)
            "
            <span>Google搜尋</span>
            <br>
            "
                中國銀行股份有限公司　　　(在大陸地區公司名稱)
            "
            <span id="linkMoea">
                <span>
                    "「"
                    <span>廠商英文名稱查詢(限經營出進口或買賣業務者)</span>
                    "」"
                    <div> ... </div>
                </span>
            </span>
        </td>
        
        get td text from type C cell:
        <td>
            "
                084
                食用油脂
            "
            <br>
            "
                089
                其他食品
            "
            <br>
        </td>        
        """

        if td is None:
            return ""
        
        if rule == 0:
            nodes = td.xpath('./text()')
        elif rule == 1:
            nodes = td.xpath('./descendant::text()')
        elif rule == 5:
            nodes = td.xpath('./text()')            
        
        text = list()
        for t in nodes:
            if t:
                t = self.clean_text(t)
                text.append(t)
        else:
            # 有時<br>後會是空字串，濾掉空字串
            return sp.join([t for t in text if t])
        
    def rule0(self, td):
        return self.get_td_text(td, sp = '+', rule=0)
    
    def rule1(self, td):
        return self.get_td_text(td, sp = '', rule=1)
    
    def rule2(self, td, key):
        items = list()
        trs = td.xpath('./table/tr')
        for i, tr in enumerate(trs):
            texts = tr.xpath('./td/text()')
            item = dict()
            for t in texts:
                t = self.clean_text(t)
                sub_item = t.split(':')
                if len(sub_item) == 1:
                    item[key]=sub_item[0]
                else:
                    k, v = sub_item
                    item[k]=v
            else:
                items.append(item)
                
        return items
        
    def rule5(self, td):
        rule5_str = self.get_td_text(td, sp = '+', rule=5)
        
        return rule5_str
    
    # 沒用到
    def retrive_tr(self, res, trcheck, info1, info2):
        selector = etree.HTML(res.content)
        divs = selector.xpath('//div[@class="tab-content"]/div')
        div0 = divs[0]
        trs = div0.xpath('./div[@class="table-responsive"]/table[@class="table table-striped"]/tbody/tr')
        
        trset = set()
        for tr in trs:
            tds = tr.xpath('./td')
            td0 = tds[0]
            key = self.get_td_text(td0)
            
            if key not in trcheck:
                self.tasklog.log(mode = 'manual', in_log = self.logstr('new_tr', info1, info2))
            
            trset.add(key)        
        return trset
    
    def logstr(self, mode, info1, info2):
        info1_str = ' '.join([k+' '+v for k, v in list(info1.items())])
        info2_str = ' '.join(['page', info2['page'], 'item', info2['item']])
        return ' '.join([mode, '@', info2_str, 'where', info1_str])
    
    
    # 每個欄位會是一個<tr>, 而正規來說<tr>內會有兩個<td>
    # 第一個是欄位名稱，第二個是欄位內容
    # 以下就是將整個schema的每個<tr>都先取出第一個<td>(欄位名稱)
    # 再取出第二個<td>(欄位內容)
    # 並新增 欄位名稱:欄位內容的key-value pair到self.schema這個dict
    def parser(self, res, info1, info2):
        selector = etree.HTML(res.content)
        divs = selector.xpath('//div[@class="tab-content"]/div')
        div0 = divs[0]
        trs = div0.xpath('./div[@class="table-responsive"]/table[@class="table table-striped"]/tbody/tr')

        for tr in trs:
            tds = tr.xpath('./td')

            td0 = tds[0]
            key = self.get_td_text(td0)
            # 解析出來的tr若只有一欄，不需要parse
            
            if len(tds) > 1:
                # 先拿第一個td
                td1 = tds[1]
                
                # 如果tr的項目名稱不在預設的欄位，將它擴充(預設rule0)，記得要處理col_book跟rule_book
                if key not in parser_cmpy_type.col_book[self.h3_query]:
                    parser_cmpy_type.col_book[self.h3_query].append(key)
                    td1_first_child = td1.xpath('./child::*')
                    if len(td1_first_child) == 0:
                        (parser_cmpy_type.rule_book[self.h3_query])[key] = 'rule0'
                    elif td1_first_child[0].tag == 'table':
                        (parser_cmpy_type.rule_book[self.h3_query])[key] = 'rule2'
                    elif td1_first_child[0].tag == 'a':
                        (parser_cmpy_type.rule_book[self.h3_query])[key] = 'rule1'
                    else:
                        (parser_cmpy_type.rule_book[self.h3_query])[key] = 'rule0'
                        

                    # 做個log
                    #self.log('new_tr', info1, info2)
                    self.tasklog.log(mode = 'manual', in_log = self.logstr('new_tr', info1, info2))
                
                # parse                 
                rule = self.tr_rule[key]
                if rule == 'rule2':
                    value = self.td_rule_handler[rule](td1, key)
                    self.data_schema[key] = value
                              
                else:
                    value = self.td_rule_handler[rule](td1)
                    self.data_schema[key] = value
                    
            # 只有一個tr的做個log
            else:
                # 寫個information
                #self.log('single_col_tr', info1, info2)
                self.tasklog.log(mode = 'manual', in_log = self.logstr('single_col_tr', info1, info2))
                
        else:
            self.data_schema['update_time'] = time.strftime('%Y/%m/%d %H:%M:%S', time.localtime())

In [9]:
# In[7]:

import pickle
import proxypool
# usage:
# cralwer_v10x2.py 3        /usr/local/bin/phantomjs 60005            3
#                  job號碼   phontomjs執行檔位置        查詢流水號，可接關  proxy_tick

# 真正要直接跑script時要用 '##'的程式碼，並把'# ===='之間的程式碼註解掉
# 真正要直接跑script時別忘了要把最後一行的break拿掉
## tasknum = sys.argv[1]
## path_phantomjs = sys.argv[2]

# ====
tasknum = 1
path_phantomjs = '/usr/local/Cellar/phantomjs/2.1.1/bin/phantomjs'
# ====


task_dir = './task_ini/'
task = pickle.load(open(task_dir+'instance-g{}_v10.x2_job.pkl'.format(tasknum), 'rb'))
taskstart = int(sys.argv[3]) if len(sys.argv) >= 4 else int(task[0][0])

crawler = cmpyinfo_crawler(path_phantomjs = path_phantomjs, logname='instance{}_v10.x2_job.log'.format(tasknum), sleep_scale='none')
crawler.proxypool.proxy_set_max = 150
crawler.proxypool.group_proxy()
crawler.proxypool.none_freq = 4

for t in task:

    
    crawler.qryCond = t[1]
    crawler.qryType = t[2]
    crawler.pageStart = t[3]
    crawler.pageEnd = t[4]
    if int(t[0]) < taskstart:
        continue
        
    print("======================================")
    print("task ", t[0], ": ", t[1], "@", t[2])
    print("======================================")
    
    crawler.search_error_cnt = 0
    crawler.set_form_data_url1(mode = 0, currentPage = 1)
    while not crawler.first_connection() and crawler.search_error_cnt < 6:
        crawler.change_proxy()

    if not crawler.search_error_cnt < 6:
        crawler.search_error_cnt = 0
        continue
    crawler.search_error_cnt = 0
    #time.sleep(random.choice([5,5.5,6,7,10,3,5,4,7,7,1]))
    crawler.resolve_page()
    crawler.parse_and_gen_schema(1, crawler.totalPage)
    
    break

task  1 :  00087006 @ ['cmpyType', 'brCmpyType', 'busmType', 'factType', 'lmtdType']
1筆、分1頁
   							
1、1
self.h3_text resolved:  公司基本資料
self.h3_query resolved:  Cmpy
Cmpy {'所營事業資料': 'G101061汽車貨運業+G101031計程車客運業+ZZ99999除許可業務外，得經營法令非禁止或限制之業務', '登記機關': '經濟部中部辦公室', '最後核准變更日期': '106年02月10日', '公司名稱': '永徠交通股份有限公司', '實收資本額(元)': '25,000,000', '核准設立日期': '056年11月20日', '公司所在地': '基隆市中正區中正路40之3號4樓', 'update_time': '2018/01/24 08:59:20', '公司狀況': '核准設立', '統一編號': '00087006', '代表人姓名': '黃秋桂', '資本總額(元)': '25,000,000'}
page 1 total 1  time    0 days  0: 2:38 round time    0 days  0: 0: 1
flush results to pandas pages:1 items:1
end of parse_and_gen_schema() pages:1 items:1


In [10]:
# In[7]:


# 真正要直接跑script時要用 '##'的程式碼，並把'# ===='之間的程式碼註解掉
#path_phantomjs = sys.argv[1]

# ====
path_phantomjs = '/usr/local/Cellar/phantomjs/2.1.1/bin/phantomjs'
# ====

task = [
        ('合夥', ['lmtdType'], 1, 1),   #0 12筆、分1頁
        ('合作社', ['factType'], 1, 5), #1 94筆、分5頁
        ('服務站', ['busmType'], 1, 5),  #2 95筆、分5頁
        ('賣場', ['busmType'], 1, 7),    #3 122筆、分7頁
        ('組', ['busmType'], 1, 8),     #4 154筆、分8頁
        ('公司', ['busmType'], 1, 8),   #5 148筆、8頁 # 這個案例經典 # done    
        ('超商', ['busmType'], 1, 32),    #6 629筆、分32頁
        ('處', ['busmType'], 1, 107),     #7 2131筆、分107頁
        ('便當', ['busmType'], 1, 114),   #8 2271筆、分114頁
        ('自助餐', ['busmType'], 1, 157), #9 3138筆、分157頁
        ('所', ['busmType'], 1, 425),     #10 8491筆、分425頁
        ('部', ['busmType'], 1, 574),     #11 11476筆、分574頁
        ('記',['busmType'], 1, 848),      #12 16957筆、分848頁
        ('院', ['busmType'], 1, 799),     #13 15976筆、分799頁
        ('工廠', ['factType'], 1, 1535),  #14 30687筆、分1535頁
        ('小吃', ['busmType'], 1, 2090),   #15 41786筆、分2090頁
        ('廠', ['busmType'], 1, 2252),     #16 45022筆、分2252頁
        ('號',['busmType'], 1, 3506),      #17 70115筆、分3506頁
        ('公司', ['brCmpyType'], 1, 4094), #18 81879筆、分4094頁
        ('廠', ['factType'], 1, 7502),    #19 150026筆、分7502頁
        ('公司', ['factType'], 1, 13860),   #20 277189筆、分13860頁
        ('店',['busmType'], 1, 16364),      #21 327277筆、分16364頁
        ('社', ['busmType'], 1, 19201),     # 384017筆、分19201頁
        ('公司', ['cmpyType'], 1, 89180),   # 1783583筆、分89180頁
        ('行',['busmType'], 1, 41233),      # 824642筆、分41233頁
        ] 



for k, t in enumerate(task):
    

    print("======================================")
    print("task ", k, ": ", t[0], "@", t[1], "From page", t[2], "to", t[3])
    print("======================================")
    crawler = cmpyinfo_crawler(t[0], qryType = t[1], pageStart=0, pageEnd=0, path_phantomjs = path_phantomjs, logname='total_job.log')
    crawler.proxy_tick = 20
    
    crawler.qryCond = t[0]
    crawler.qryType = t[1]
    crawler.set_form_data_url1(mode = 0, currentPage = 1)

    
    crawler.first_connection()
    crawler.resolve_page()
    crawler.pageStart = t[2]
    crawler.pageEnd = t[3]
    crawler.parse_and_gen_schema(crawler.pageStart, 500 if crawler.totalPage > 500 else crawler.totalPage)
    crawler.session.close()
    
    break

task  0 :  合夥 @ ['lmtdType'] From page 1 to 1
17筆、分1頁
   							
17、1
self.h3_text resolved:  有限合夥登記基本資料
self.h3_query resolved:  Lmtd
Lmtd {'實收出資額(元)': '500,000', '登記機關': '經濟部中部辦公室', '有限合夥人': [{'有限合夥人': '(共2人)', '出資額(元)': '200,000', '責任類型': '有限責任'}], '最近一次登記狀況核准日期及文號': '106年02月20日(1063570004號)', '現況': '核准設立', '存續期間': '自起至止', '核准設立日期': '106年02月20日(1063570004號)', 'update_time': '2018/01/24 09:06:10', '約定解散事由': '', '普通合夥人姓名': [{'出資額(元)': '300,000', '普通合夥人姓名': '王亭雅', '責任類型': '無限責任'}], '有限合夥名稱': '雅文有限合夥', '統一編號': '42904319', '代表人姓名': [{'代表人姓名': '王亭雅', '出資額(元)': '0', '責任類型': '無限責任'}], '所在地': '臺南市麻豆區大埕里忠孝路25之3號一樓', '經理人姓名': '', '所營事業項目': '1H201010一般投資業+2ZZ99999除許可業務外，得經營法令非禁止或限制之業務'}
self.h3_text resolved:  有限合夥登記基本資料
self.h3_query resolved:  Lmtd
Lmtd {'實收出資額(元)': '800,010,000', '登記機關': '經濟部中部辦公室', '有限合夥人': [{'有限合夥人': '(共1人)', '出資額(元)': '800,000,000', '責任類型': '有限責任'}], '最近一次登記狀況核准日期及文號': '106年06月28日(10635700190號)', '現況': '核准設立', '存續期間': '自106年03月24日起至136年03月23日止', '核准設立日期': '106年06月28日(1063